# Econometria Espacial com PanelBox

Um guia completo para análise de dados em painel com dependência espacial.

## Índice

1. [Introdução](#intro)
2. [Setup e Imports](#setup)
3. [Carregamento de Dados](#data)
4. [Matriz de Pesos Espaciais](#weights)
5. [Diagnósticos Espaciais](#diagnostics)
6. [Estimação de Modelos Espaciais](#models)
7. [Comparação de Modelos](#comparison)
8. [Decomposição de Efeitos](#effects)
9. [Diagnósticos Pós-Estimação](#post-diagnostics)
10. [Inferência Robusta](#robust)
11. [Interpretação Econômica](#interpretation)
12. [Exportação de Resultados](#export)

---

<a id='intro'></a>
## 1. Introdução

### O que é autocorrelação espacial?

Autocorrelação espacial ocorre quando observações geograficamente próximas são mais similares do que seria esperado por acaso. Em econometria, isso viola a suposição de independência dos erros.

### Por que modelos espaciais são necessários?

- **Spillovers econômicos:** Políticas ou choques em uma região afetam regiões vizinhas
- **Difusão tecnológica:** Inovações se espalham geograficamente
- **Mercados regionais:** Preços e salários em regiões próximas são correlacionados
- **Dependência não-observada:** Fatores omitidos com padrão espacial

### Quando usar SAR vs SEM vs SDM?

- **SAR (Spatial Autoregressive):** Quando há spillovers na variável dependente
- **SEM (Spatial Error Model):** Quando há autocorrelação espacial nos erros
- **SDM (Spatial Durbin Model):** Quando há spillovers tanto em y quanto em X

<a id='setup'></a>
## 2. Setup e Imports

In [ ]:
# Imports principais
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

# PanelBox
import panelbox as pb

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Warnings
import warnings
warnings.filterwarnings('ignore')

print(f"PanelBox version: {pb.__version__}")

<a id='data'></a>
## 3. Carregamento de Dados

Vamos usar um dataset de preços de imóveis nos estados dos EUA (1990-2020) com informações geográficas.

In [ ]:
# Simular dados para exemplo (em produção, carregue dados reais)
np.random.seed(42)

# Criar dados em painel
states = ['CA', 'TX', 'FL', 'NY', 'PA', 'IL', 'OH', 'GA', 'NC', 'MI']
years = range(2010, 2021)
n_states = len(states)
n_years = len(years)

# Gerar dados
data = []
for state in states:
    for year in years:
        data.append({
            'state': state,
            'year': year,
            'house_price': np.random.uniform(200000, 600000) + (year - 2010) * 10000,
            'income': np.random.uniform(50000, 100000) + (year - 2010) * 2000,
            'population': np.random.uniform(1e6, 10e6),
            'unemployment': np.random.uniform(3, 8),
            'interest_rate': np.random.uniform(3, 5)
        })

df = pd.DataFrame(data)

# Adicionar coordenadas (simplificado)
state_coords = {
    'CA': (-119.4179, 36.7783),
    'TX': (-99.9018, 31.9686),
    'FL': (-81.5158, 27.6648),
    'NY': (-75.0000, 43.0000),
    'PA': (-77.1945, 41.2033),
    'IL': (-89.0000, 40.0000),
    'OH': (-82.9071, 40.4173),
    'GA': (-82.9001, 32.1656),
    'NC': (-79.0193, 35.7596),
    'MI': (-85.5226, 44.3148)
}

df['longitude'] = df['state'].map(lambda s: state_coords[s][0])
df['latitude'] = df['state'].map(lambda s: state_coords[s][1])

print(f"Dataset shape: {df.shape}")
print(f"States: {n_states}")
print(f"Years: {n_years}")
print(f"\nFirst few rows:")
df.head()

### Visualização Espacial dos Dados

In [ ]:
# Visualizar distribuição espacial (média por estado)
avg_by_state = df.groupby('state').agg({
    'house_price': 'mean',
    'longitude': 'first',
    'latitude': 'first'
}).reset_index()

fig, ax = plt.subplots(1, 1, figsize=(10, 8))

scatter = ax.scatter(
    avg_by_state['longitude'],
    avg_by_state['latitude'],
    c=avg_by_state['house_price'],
    s=200,
    cmap='RdYlBu_r',
    edgecolor='black',
    linewidth=1
)

# Adicionar labels
for idx, row in avg_by_state.iterrows():
    ax.annotate(
        row['state'],
        (row['longitude'], row['latitude']),
        fontsize=10,
        ha='center'
    )

plt.colorbar(scatter, label='Average House Price ($)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Average House Prices by State')
plt.tight_layout()
plt.show()

<a id='weights'></a>
## 4. Matriz de Pesos Espaciais

A matriz W define a estrutura de vizinhança entre as unidades espaciais.

In [ ]:
# Método 1: Matriz baseada em distância
coords = avg_by_state[['longitude', 'latitude']].values
W_distance = pb.SpatialWeights.from_distance(
    coords,
    threshold=1500  # 1500 km threshold
)

print(f"Distance-based W shape: {W_distance.W.shape}")
print(f"Average number of neighbors: {W_distance.W.sum(axis=1).mean():.2f}")

In [ ]:
# Método 2: k-nearest neighbors
W_knn = pb.SpatialWeights.from_knn(coords, k=3)

print(f"k-NN W shape: {W_knn.W.shape}")
print(f"Number of neighbors per unit: {W_knn.W.sum(axis=1)[0]:.0f}")

In [ ]:
# Row-standardize the weight matrix
W = W_distance.standardize('row')

# Verificar propriedades
print("Weight matrix properties:")
print(f"- Dimensions: {W.W.shape}")
print(f"- Row sums (should be 1): {W.W.sum(axis=1)[:5]}")
print(f"- Sparsity: {(W.W == 0).sum() / W.W.size:.2%}")
print(f"- Symmetric: {np.allclose(W.W, W.W.T)}")

### Visualizar Matriz de Pesos

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Matriz W
im1 = ax1.imshow(W.W, cmap='YlOrRd', aspect='equal')
ax1.set_title('Spatial Weight Matrix W')
ax1.set_xlabel('State j')
ax1.set_ylabel('State i')
plt.colorbar(im1, ax=ax1)

# Plot 2: Conectividade
connectivity = (W.W > 0).astype(int)
im2 = ax2.imshow(connectivity, cmap='binary', aspect='equal')
ax2.set_title('Connectivity Structure')
ax2.set_xlabel('State j')
ax2.set_ylabel('State i')

plt.tight_layout()
plt.show()

<a id='diagnostics'></a>
## 5. Diagnósticos Espaciais

Antes de estimar modelos espaciais, precisamos testar se há autocorrelação espacial.

In [ ]:
# Criar experimento
experiment = pb.PanelExperiment(
    data=df,
    formula="house_price ~ income + population + unemployment + interest_rate",
    entity_col="state",
    time_col="year"
)

# Estimar OLS como baseline
ols_result = experiment.fit_model('pooled_ols', name='ols')
print(ols_result.summary())

### Teste de Moran's I

In [ ]:
# Teste de Moran's I nos resíduos OLS
morans_test = pb.MoranIPanelTest(
    ols_result.resid,
    W,
    df['state'],
    df['year']
)

morans_result = morans_test.run()

print("Moran's I Test for Spatial Autocorrelation")
print("="*45)
print(f"Moran's I statistic: {morans_result.statistic:.4f}")
print(f"Expected value:      {morans_result.expected:.4f}")
print(f"Variance:            {morans_result.variance:.6f}")
print(f"z-statistic:         {morans_result.z_stat:.4f}")
print(f"p-value:             {morans_result.pvalue:.4f}")
print("\nInterpretation:")
if morans_result.pvalue < 0.05:
    print("⚠️  Significant spatial autocorrelation detected!")
    print("   → Spatial models are needed")
else:
    print("✓ No significant spatial autocorrelation")
    print("   → OLS may be sufficient")

### Visualizar Moran's I por Período

In [ ]:
# Calcular Moran's I para cada ano
morans_by_year = []
for year in years:
    year_data = df[df['year'] == year]
    year_resid = ols_result.resid[df['year'] == year]
    
    test = pb.MoranIPanelTest(
        year_resid,
        W,
        year_data['state'],
        year_data['year'],
        method='single'
    )
    result = test.run()
    
    morans_by_year.append({
        'year': year,
        'morans_i': result.statistic,
        'pvalue': result.pvalue
    })

morans_df = pd.DataFrame(morans_by_year)

# Plotar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Moran's I ao longo do tempo
ax1.plot(morans_df['year'], morans_df['morans_i'], marker='o')
ax1.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax1.set_xlabel('Year')
ax1.set_ylabel("Moran's I")
ax1.set_title("Moran's I Over Time")
ax1.grid(True, alpha=0.3)

# P-values
ax2.plot(morans_df['year'], morans_df['pvalue'], marker='o', color='orange')
ax2.axhline(y=0.05, color='r', linestyle='--', alpha=0.5, label='α=0.05')
ax2.set_xlabel('Year')
ax2.set_ylabel('p-value')
ax2.set_title('Significance of Spatial Autocorrelation')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### LM Tests para Especificação do Modelo

In [ ]:
# Executar bateria de testes LM
lm_results = pb.run_lm_tests(ols_result, W)

print("Lagrange Multiplier Tests for Model Specification")
print("="*50)
print("\nTest Results:")
print("-"*50)

tests = ['lm_lag', 'lm_error', 'robust_lm_lag', 'robust_lm_error']
for test_name in tests:
    result = lm_results[test_name]
    print(f"\n{test_name.upper().replace('_', ' ')}:")
    print(f"  Statistic: {result.statistic:8.4f}")
    print(f"  p-value:   {result.pvalue:8.4f}")
    print(f"  Significant: {'Yes ***' if result.pvalue < 0.01 else 'Yes **' if result.pvalue < 0.05 else 'Yes *' if result.pvalue < 0.10 else 'No'}")

print("\n" + "="*50)
print(f"\nRECOMMENDATION: {lm_results['recommendation']}")
print("\nInterpretation:")
if lm_results['recommendation'] == 'SAR':
    print("→ Use Spatial Lag Model (SAR)")
    print("  Evidence of spatial spillovers in the dependent variable")
elif lm_results['recommendation'] == 'SEM':
    print("→ Use Spatial Error Model (SEM)")
    print("  Evidence of spatial autocorrelation in the error term")
elif lm_results['recommendation'] == 'SDM':
    print("→ Use Spatial Durbin Model (SDM)")
    print("  Evidence of both spatial lag and spatial error effects")
else:
    print("→ No spatial effects detected, OLS may be appropriate")

<a id='models'></a>
## 6. Estimação de Modelos Espaciais

### SAR - Spatial Autoregressive Model

In [ ]:
# Estimar SAR com efeitos fixos
sar = pb.SpatialLag(
    formula="house_price ~ income + population + unemployment + interest_rate",
    data=df,
    entity_col="state",
    time_col="year",
    W=W
)

sar_result = sar.fit(effects='entity')
print(sar_result.summary())

print(f"\nSpatial lag parameter ρ = {sar_result.rho:.4f}")
print(f"Interpretation: A 1% increase in neighbors' house prices")
print(f"                leads to a {sar_result.rho:.2%} increase in own house price")

### SEM - Spatial Error Model

In [ ]:
# Estimar SEM com efeitos fixos
sem = pb.SpatialError(
    formula="house_price ~ income + population + unemployment + interest_rate",
    data=df,
    entity_col="state",
    time_col="year",
    W=W
)

sem_result = sem.fit(effects='entity')
print(sem_result.summary())

print(f"\nSpatial error parameter λ = {sem_result.lambda_:.4f}")
print(f"Interpretation: Spatial clustering in unobserved factors")

### SDM - Spatial Durbin Model

In [ ]:
# Estimar SDM com efeitos fixos
sdm = pb.SpatialDurbin(
    formula="house_price ~ income + population + unemployment + interest_rate",
    data=df,
    entity_col="state",
    time_col="year",
    W=W
)

sdm_result = sdm.fit(effects='entity')
print(sdm_result.summary())

print(f"\nSpatial lag parameter ρ = {sdm_result.rho:.4f}")
print("\nSpatial lag of X coefficients (θ):")
for var, coef in sdm_result.theta.items():
    print(f"  W*{var}: {coef:.4f}")

<a id='comparison'></a>
## 7. Comparação de Modelos

In [ ]:
# Comparar todos os modelos
models = {
    'OLS': ols_result,
    'SAR-FE': sar_result,
    'SEM-FE': sem_result,
    'SDM-FE': sdm_result
}

comparison = pd.DataFrame({
    'Log-Likelihood': [m.llf for m in models.values()],
    'AIC': [m.aic for m in models.values()],
    'BIC': [m.bic for m in models.values()],
    'R-squared': [m.rsquared if hasattr(m, 'rsquared') else np.nan for m in models.values()],
    'Spatial Param': [
        np.nan,
        sar_result.rho,
        sem_result.lambda_,
        sdm_result.rho
    ]
}, index=models.keys())

print("Model Comparison")
print("="*60)
print(comparison.round(4))

# Identificar melhor modelo
best_aic = comparison['AIC'].idxmin()
best_bic = comparison['BIC'].idxmin()

print(f"\nBest model by AIC: {best_aic}")
print(f"Best model by BIC: {best_bic}")

### Visualizar Comparação

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# AIC comparison
ax1 = axes[0]
bars1 = ax1.bar(comparison.index, comparison['AIC'], color='steelblue')
bars1[comparison['AIC'].argmin()].set_color('darkgreen')
ax1.set_ylabel('AIC')
ax1.set_title('AIC Comparison (lower is better)')
ax1.grid(True, alpha=0.3)

# BIC comparison
ax2 = axes[1]
bars2 = ax2.bar(comparison.index, comparison['BIC'], color='coral')
bars2[comparison['BIC'].argmin()].set_color('darkgreen')
ax2.set_ylabel('BIC')
ax2.set_title('BIC Comparison (lower is better)')
ax2.grid(True, alpha=0.3)

# Log-likelihood
ax3 = axes[2]
bars3 = ax3.bar(comparison.index, comparison['Log-Likelihood'], color='purple')
bars3[comparison['Log-Likelihood'].argmax()].set_color('darkgreen')
ax3.set_ylabel('Log-Likelihood')
ax3.set_title('Log-Likelihood (higher is better)')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<a id='effects'></a>
## 8. Decomposição de Efeitos (SDM)

No modelo SDM, os coeficientes β não representam os efeitos marginais totais. Precisamos decompor em efeitos diretos, indiretos e totais.

In [ ]:
# Calcular decomposição de efeitos para SDM
effects = pb.compute_spatial_effects(sdm_result, W)

print("Effects Decomposition (SDM)")
print("="*60)
print("\nDirect Effects (impact on own region):")
for var, effect in effects['direct'].items():
    print(f"  {var:20s}: {effect['estimate']:8.4f} (SE: {effect['std_error']:.4f})")

print("\nIndirect Effects (spillover to neighbors):")
for var, effect in effects['indirect'].items():
    print(f"  {var:20s}: {effect['estimate']:8.4f} (SE: {effect['std_error']:.4f})")

print("\nTotal Effects (direct + indirect):")
for var, effect in effects['total'].items():
    print(f"  {var:20s}: {effect['estimate']:8.4f} (SE: {effect['std_error']:.4f})")

# Calcular proporção dos efeitos indiretos
print("\nProportion of Indirect Effects:")
for var in effects['direct'].keys():
    if effects['total'][var]['estimate'] != 0:
        pct_indirect = (effects['indirect'][var]['estimate'] / 
                       effects['total'][var]['estimate'] * 100)
        print(f"  {var:20s}: {pct_indirect:5.1f}%")

### Visualizar Decomposição de Efeitos

In [ ]:
# Preparar dados para visualização
variables = list(effects['direct'].keys())
direct_effects = [effects['direct'][v]['estimate'] for v in variables]
indirect_effects = [effects['indirect'][v]['estimate'] for v in variables]
total_effects = [effects['total'][v]['estimate'] for v in variables]

# Criar gráfico de barras empilhadas
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(variables))
width = 0.25

bars1 = ax.bar(x - width, direct_effects, width, label='Direct', color='steelblue')
bars2 = ax.bar(x, indirect_effects, width, label='Indirect', color='coral')
bars3 = ax.bar(x + width, total_effects, width, label='Total', color='green')

ax.set_xlabel('Variables')
ax.set_ylabel('Effect Size')
ax.set_title('Decomposition of Spatial Effects (SDM Model)')
ax.set_xticks(x)
ax.set_xticklabels(variables, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

<a id='post-diagnostics'></a>
## 9. Diagnósticos Pós-Estimação

In [ ]:
# Testar autocorrelação espacial nos resíduos do modelo SAR
morans_sar = pb.MoranIPanelTest(
    sar_result.resid,
    W,
    df['state'],
    df['year']
)

morans_sar_result = morans_sar.run()

print("Post-Estimation Diagnostics: SAR Model")
print("="*45)
print(f"Moran's I (residuals): {morans_sar_result.statistic:.4f}")
print(f"p-value:               {morans_sar_result.pvalue:.4f}")

if morans_sar_result.pvalue > 0.05:
    print("\n✓ No remaining spatial autocorrelation")
    print("  The SAR model successfully captured spatial dependence")
else:
    print("\n⚠️  Spatial autocorrelation remains in residuals")
    print("  Consider SDM or GNS model")

### Local Indicators of Spatial Association (LISA)

In [ ]:
# Calcular LISA para identificar clusters locais
# Usar resíduos médios por estado
avg_resid_by_state = pd.DataFrame({
    'state': df['state'].unique(),
    'avg_residual': [sar_result.resid[df['state'] == s].mean() 
                    for s in df['state'].unique()]
})

lisa = pb.LocalMoranI(
    avg_resid_by_state['avg_residual'].values,
    W
)

lisa_results = lisa.run()

# Classificar clusters
avg_resid_by_state['lisa_stat'] = lisa_results.Is
avg_resid_by_state['lisa_pval'] = lisa_results.p_values
avg_resid_by_state['cluster_type'] = lisa_results.classify_clusters(
    avg_resid_by_state['avg_residual'].values
)

print("Local Moran's I Results:")
print(avg_resid_by_state[['state', 'lisa_stat', 'lisa_pval', 'cluster_type']])

# Interpretar clusters
print("\nCluster Types:")
print("- HH: High values surrounded by high values (hot spots)")
print("- LL: Low values surrounded by low values (cold spots)")
print("- HL: High values surrounded by low values (spatial outliers)")
print("- LH: Low values surrounded by high values (spatial outliers)")
print("- NS: Not significant")

<a id='robust'></a>
## 10. Inferência Robusta

Usar Spatial HAC (Conley 1999) para erros padrão robustos a autocorrelação espacial e temporal.

In [ ]:
# Re-estimar SAR com Spatial HAC
sar_hac = pb.SpatialLag(
    formula="house_price ~ income + population + unemployment + interest_rate",
    data=df,
    entity_col="state",
    time_col="year",
    W=W
)

sar_hac_result = sar_hac.fit(
    effects='entity',
    se_type='spatial_hac',
    spatial_cutoff=1500,  # km
    temporal_cutoff=2      # years
)

# Comparar erros padrão
se_comparison = pd.DataFrame({
    'Variable': sar_result.params.index,
    'Coef': sar_result.params.values,
    'SE (Standard)': sar_result.bse.values,
    'SE (Spatial HAC)': sar_hac_result.bse.values
})

se_comparison['SE Ratio'] = (se_comparison['SE (Spatial HAC)'] / 
                             se_comparison['SE (Standard)'])

print("Standard Errors Comparison")
print("="*60)
print(se_comparison.round(4))
print("\nInterpretation:")
print("SE Ratio > 1: Spatial HAC SEs are larger (more conservative)")
print("SE Ratio < 1: Spatial HAC SEs are smaller")

<a id='interpretation'></a>
## 11. Interpretação Econômica

### Principais Insights dos Modelos Espaciais

In [ ]:
print("ECONOMIC INTERPRETATION OF SPATIAL MODELS")
print("="*60)

print("\n1. SPATIAL SPILLOVERS (from SAR model):")
print(f"   - Spatial lag coefficient ρ = {sar_result.rho:.3f}")
print(f"   - Interpretation: A 10% increase in neighboring states' house prices")
print(f"     leads to a {sar_result.rho * 10:.1f}% increase in own state's prices")
print(f"   - This suggests strong regional housing market integration")

print("\n2. DIRECT vs INDIRECT EFFECTS (from SDM model):")
for var in ['income']:
    if var in effects['direct']:
        direct = effects['direct'][var]['estimate']
        indirect = effects['indirect'][var]['estimate']
        total = effects['total'][var]['estimate']
        
        print(f"\n   Variable: {var}")
        print(f"   - Direct effect:   {direct:.2f}")
        print(f"     (1% increase in own state's {var} → ${direct:.0f} price increase)")
        print(f"   - Indirect effect: {indirect:.2f}")
        print(f"     (1% increase in neighbors' {var} → ${indirect:.0f} price increase)")
        print(f"   - Total effect:    {total:.2f}")
        print(f"   - Spillover ratio: {indirect/total*100:.1f}% of total effect is spillover")

print("\n3. POLICY IMPLICATIONS:")
print("   - Local policies have regional effects through spatial spillovers")
print("   - Need for coordinated regional policies")
print("   - Economic shocks propagate across neighboring regions")
print("   - Housing markets are interconnected across state boundaries")

print("\n4. MODEL SELECTION:")
print(f"   - Best model (by AIC): {best_aic}")
if 'SDM' in best_aic:
    print("   - SDM captures both endogenous and exogenous spatial interactions")
elif 'SAR' in best_aic:
    print("   - SAR indicates spatial dependence in the dependent variable")
elif 'SEM' in best_aic:
    print("   - SEM suggests spatial patterns in unobserved factors")

<a id='export'></a>
## 12. Exportação de Resultados

### Exportar para LaTeX

In [ ]:
# Criar tabela LaTeX com resultados
latex_table = sar_result.to_latex(
    caption="Spatial Autoregressive Model Results",
    label="tab:sar_results",
    stars=True
)

print("LaTeX Table (first 20 lines):")
print(latex_table[:800])

# Salvar em arquivo
with open('sar_results.tex', 'w') as f:
    f.write(latex_table)
print("\n✓ LaTeX table saved to 'sar_results.tex'")

### Gerar Relatório HTML

In [ ]:
# Gerar relatório HTML completo
report_path = experiment.generate_spatial_report('spatial_analysis_report.html')
print(f"✓ HTML report saved to '{report_path}'")
print("\nReport includes:")
print("  - Model comparison table")
print("  - Spatial diagnostics (Moran's I, LM tests)")
print("  - Model results with spatial parameters")
print("  - Effects decomposition (for SDM)")
print("  - Interactive visualizations")
print("  - Residual diagnostics")

## Conclusão

Este tutorial cobriu o workflow completo de econometria espacial com PanelBox:

✅ **Diagnósticos Espaciais:** Moran's I e LM tests para detectar autocorrelação espacial  
✅ **Modelos Espaciais:** SAR, SEM, SDM com efeitos fixos/aleatórios  
✅ **Decomposição de Efeitos:** Efeitos diretos, indiretos e totais  
✅ **Inferência Robusta:** Spatial HAC para erros padrão robustos  
✅ **Visualizações:** Mapas, gráficos de diagnóstico e comparações  
✅ **Exportação:** LaTeX, HTML, e integração com relatórios  

### Próximos Passos

1. **Dados Reais:** Aplicar a dados geográficos reais com shapefiles
2. **Matrizes W Alternativas:** Experimentar diferentes especificações de vizinhança
3. **Modelos Dinâmicos:** Adicionar lags temporais aos modelos espaciais
4. **Cross-Validation:** Validar previsões out-of-sample
5. **Modelos Não-Lineares:** Explorar modelos espaciais para variáveis categóricas

### Recursos Adicionais

- [PanelBox Documentation](https://panelbox.readthedocs.io)
- [Spatial Econometrics Theory Guide](https://panelbox.readthedocs.io/spatial/theory)
- [Examples Gallery](https://panelbox.readthedocs.io/examples/spatial)

---

**PanelBox** - The most complete spatial econometrics toolkit for Python 🚀